# Model Processing

## Imports & General Settings 

In [13]:
import unittest

from pycwt import Morlet
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm

# Our imports
from data import WaveletTransform, AFECGDataset, SecondDataset, WrapperDataset
import dsp
from model.blocks import ConvNet, BRNN, SoftmaxAttention
from model.baseline import Baseline
from training import train
import utils


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## Dataset creation

In [15]:
dataset_name = 'afdb'
dataset = AFECGDataset(dataset_name, '../data/files/', transform=True)

In [ ]:
dataset.load()

In [36]:
dataset = SecondDataset(dataset_name, '../data/files/', wt=WaveletTransform(Morlet(6), resample=20))
class_weights = dataset.load('./backup')

Loaded 2300 samples from backup
torch.Size([2300])
torch.Size([2300, 20, 375])


In [37]:
true_examples = dataset.samples[dataset.labels == 1]
false_examples = dataset.samples[dataset.labels == 0]
false_labels = dataset.labels[dataset.labels == 0]
augmented = []
augmented_labels = []
for i in range(len(false_examples) // len(true_examples)):
    augmented.append(true_examples.clone())
    augmented_labels.append(torch.ones(len(true_examples)).long())

print(torch.cat(augmented).shape)
print(false_examples.shape)
dataset.samples = torch.cat([torch.cat(augmented), false_examples])
dataset.labels = torch.cat([torch.cat(augmented_labels), false_labels])

torch.Size([2268, 20, 375])
torch.Size([2282, 20, 375])


In [38]:
print(dataset.labels.sum())
print(len(dataset) - dataset.labels.sum())

tensor(2268)
tensor(2282)


## Training

In [ ]:
heldout = int(len(dataset) * 0.2)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - heldout, heldout])

# model = Baseline(add_brnn=True, hidden_size=100)
model = nn.Sequential(
    ConvNet(size=(375, 20), batch=False),
    nn.Linear(50, 2)
)

config = dict(
    num_workers=8,
    batch_size=90,
    learning_rate=0.0005,
    weight_decay=0.01,
    class_weights=None,
    num_epochs=200,
    is_notebook=True
)

train(model, dataset, config)

Loss:  tensor(0.6918, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1])
Prediction: tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])
Correct: 51
Loss:  tensor(13.3095, grad_fn=<NllLossBackward>)
Loss:  tensor(0.7623, grad_fn=<NllLossBackward>)
Loss:  tensor(2.9569, grad_fn=<NllLossBackward>)
Loss:  tensor(3.0477, grad_fn=<NllLossBackward>)
Loss:  tensor(2.0098, grad_fn=<NllLossBackward>)
Loss:  tensor(2.3607, grad_fn=<NllLossBackward>)
Loss:  tensor(0.8188, grad_fn=<NllLossBackward>)
Loss:  tensor(1.1312, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,

Loss:  tensor(0.5631, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 1])
Prediction: tensor([0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
        0, 1])
Correct: 66
Loss:  tensor(0.5377, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5109, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5986, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4985, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5182, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5706, grad_fn=<NllLossBackward>)
Loss:  tensor(0.6101, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4711, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
        0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 

Loss:  tensor(0.5754, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
        0, 1])
Prediction: tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
        1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])
Correct: 59
Loss:  tensor(0.5316, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4642, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4653, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4779, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5290, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5447, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5346, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4499, grad_fn=<NllLossBackward>)
Ground truth: tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 

Loss:  tensor(0.4339, grad_fn=<NllLossBackward>)
Ground truth: tensor([1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
        1, 1])
Prediction: tensor([1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
        1, 1])
Correct: 76
Loss:  tensor(0.4531, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5894, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4750, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4274, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4336, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5731, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4915, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5686, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 

Loss:  tensor(0.3886, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
        0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        1, 1])
Prediction: tensor([0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        1, 1])
Correct: 74
Loss:  tensor(0.4109, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4389, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4272, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4429, grad_fn=<NllLossBackward>)
Loss:  tensor(0.3815, grad_fn=<NllLossBackward>)
Loss:  tensor(0.5293, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4221, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4370, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 

...

Loss:  tensor(0.1728, grad_fn=<NllLossBackward>)
Ground truth: tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 1])
Prediction: tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
        1, 1])
Correct: 86
Loss:  tensor(0.1425, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1738, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1905, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1344, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1723, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2055, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1851, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2136, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 

Loss:  tensor(0.1658, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 1])
Prediction: tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 1])
Correct: 87
Loss:  tensor(0.1047, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1310, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1935, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1619, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2320, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2208, grad_fn=<NllLossBackward>)
Loss:  tensor(0.3894, grad_fn=<NllLossBackward>)
Loss:  tensor(0.3140, grad_fn=<NllLossBackward>)
Ground truth: tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 

Loss:  tensor(0.1937, grad_fn=<NllLossBackward>)
Ground truth: tensor([1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0])
Prediction: tensor([1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
        0, 1])
Correct: 86
Loss:  tensor(0.1236, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1266, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0831, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1025, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1313, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1167, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2123, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2329, grad_fn=<NllLossBackward>)
Ground truth: tensor([1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
        1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 

Loss:  tensor(0.1117, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 1])
Prediction: tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 1])
Correct: 87
Loss:  tensor(0.1164, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1591, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0937, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1190, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0877, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1245, grad_fn=<NllLossBackward>)
Loss:  tensor(0.1717, grad_fn=<NllLossBackward>)
Loss:  tensor(0.0887, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 

Loss:  tensor(0.2107, grad_fn=<NllLossBackward>)
Ground truth: tensor([1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0])
Prediction: tensor([1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0])
Correct: 86
Loss:  tensor(0.1812, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2405, grad_fn=<NllLossBackward>)
Loss:  tensor(0.3359, grad_fn=<NllLossBackward>)
Loss:  tensor(0.3429, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2806, grad_fn=<NllLossBackward>)
Loss:  tensor(0.4939, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2172, grad_fn=<NllLossBackward>)
Loss:  tensor(0.2034, grad_fn=<NllLossBackward>)
Ground truth: tensor([0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 